In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, ElasticNet, BayesianRidge
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from pathlib import Path
import joblib
import sys
import os
# I had an issue with importing functions from the src directory, so I used chatGPT to get the line below
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
import functions

In [13]:
input_path = Path.cwd()
root = input_path.parent
data_path = root / "data" / "development_final_data.csv"
train_df = pd.read_csv(data_path, header=0)
x = train_df.drop(columns=['BMI'])
y = train_df['BMI']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


In [ ]:
for model in [ElasticNet(), BayesianRidge(), SVR()]:
    print(f"Model: {model.__class__.__name__}")
    # Create a pipeline for each baseline model
    pipeline = functions.create_pipeline(model)
    preds, trained_pipeline = functions.train_model_and_predict(x_train, y_train, x_test, pipeline, filename=f"{model.__class__.__name__}.pkl", save=True)

MSE: 9.352660460676493
MAE: 2.4906351578827577
R2: 0.04225909773064784
[25.09206813 25.08976305 25.08493497 25.62562572 24.60877506 25.56461833
 25.13836911 25.39667963 24.39479904 25.27622784 25.44340145 24.46922925
 24.50057243 24.31288596 25.10701473 25.07894693 24.58116423 25.07019001
 25.07019001 26.54431363 25.24110871 25.40336276 25.3916327  24.53119328
 25.69830177 25.05066332 24.77523892 25.20966124 25.60959234 24.38446027
 25.3930582  24.85858479 25.07309233 25.0855916  25.07019001 24.65745875
 25.10009647 25.48355085 24.57066834 25.4548882  24.58509956 25.08817177
 25.17505416 24.33461829 25.35103903 25.09178026 24.30794218 24.47848037
 24.62732271]
MSE: 9.426759044357853
MAE: 2.2718814067520285
R2: 0.034671177192930225
[25.5063042  25.58381224 25.92711858 27.37319588 24.84667569 25.75645085
 25.87404325 24.72982836 23.25057918 26.85870535 24.80966813 24.51362656
 22.46527269 21.97561791 25.5696788  25.82690005 25.03112161 25.20711461
 25.61642941 26.76843843 23.99915465 24.